In [ ]:
pip install transformers accelerate


In [7]:
import requests
from PIL import Image
import torch

# ---- BLIP-1 ----
from transformers import BlipProcessor, BlipForQuestionAnswering

processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base").to("cuda")

img_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg'
raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

question = "how many humans are in the picture?"
inputs = processor(raw_image, question, return_tensors="pt").to("cuda")

out = model.generate(**inputs)
print("BLIP-1 Answer:", processor.decode(out[0], skip_special_tokens=True))


# ---- BLIP-2 with OPT-2.7B ----
# from transformers import Blip2Processor, Blip2ForConditionalGeneration
# processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
# model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b", torch_dtype=torch.float16).to("cuda")
# inputs = processor(raw_image, question, return_tensors="pt").to("cuda", torch.float16)
# out = model.generate(**inputs)
# print("BLIP-2 OPT-2.7B Answer:", processor.decode(out[0], skip_special_tokens=True))


# ---- BLIP-2 with FLAN-T5 XL ----
# from transformers import Blip2Processor, Blip2ForConditionalGeneration
# processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xl")
# model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xl", torch_dtype=torch.float16).to("cuda")
# inputs = processor(raw_image, question, return_tensors="pt").to("cuda", torch.float16)
# out = model.generate(**inputs, max_new_tokens=50)
# print("BLIP-2 FLAN-T5 XL Answer:", processor.decode(out[0], skip_special_tokens=True))


# ---- BLIP-2 with FLAN-T5 XXL ----
# from transformers import Blip2Processor, Blip2ForConditionalGeneration
# processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xxl")
# model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xxl", torch_dtype=torch.float16).to("cuda")
# inputs = processor(raw_image, question, return_tensors="pt").to("cuda", torch.float16)
# out = model.generate(**inputs, max_new_tokens=50)
# print("BLIP-2 FLAN-T5 XXL Answer:", processor.decode(out[0], skip_special_tokens=True))


# ---- BakLLaVA ----
# from transformers import AutoProcessor, AutoModelForVision2Seq
# processor = AutoProcessor.from_pretrained("LanguageBind/BakLLaVA-1")
# model = AutoModelForVision2Seq.from_pretrained("LanguageBind/BakLLaVA-1", torch_dtype=torch.float16).to("cuda")
# inputs = processor(text=question, images=raw_image, return_tensors="pt").to("cuda", torch.float16)
# out = model.generate(**inputs, max_new_tokens=50)
# print("BakLLaVA Answer:", processor.batch_decode(out, skip_special_tokens=True)[0])


# ---- Qwen-VL ----
# from transformers import AutoProcessor, AutoModelForVision2Seq
# processor = AutoProcessor.from_pretrained("Qwen/Qwen-VL")
# model = AutoModelForVision2Seq.from_pretrained("Qwen/Qwen-VL", torch_dtype=torch.float16).to("cuda")
# inputs = processor(raw_image, question, return_tensors="pt").to("cuda", torch.float16)
# out = model.generate(**inputs)
# print("Qwen-VL Answer:", processor.batch_decode(out, skip_special_tokens=True)[0])


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

1


In [6]:
!pip install rouge_score

In [8]:
!pip install bert-score rouge-score nltk sacrebleu --quiet

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [10]:
!pip install git+https://github.com/neulab/BARTScore.git --quiet

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: git+https://github.com/neulab/BARTScore.git does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [13]:
!git clone https://github.com/neulab/BARTScore.git



Cloning into 'BARTScore'...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


remote: Enumerating objects: 220, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 220 (delta 18), reused 14 (delta 14), pack-reused 194 (from 1)
Receiving objects: 100% (220/220), 101.98 MiB | 23.79 MiB/s, done.
Resolving deltas: 100% (47/47), done.
Updating files: 100% (192/192), done.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [31]:
setup_code = """
from setuptools import setup, find_packages

setup(
    name='bart_score',
    version='0.1.0',
    description='BARTScore: Evaluating Text Generation with BART',
    author='Neulab',
    url='https://github.com/neulab/BARTScore',
    packages=find_packages(),
    install_requires=[
        'torch>=1.7.0',
        'transformers>=4.0.0',
        'scipy',
        'numpy',
        'tqdm',
    ],
    classifiers=[
        'Programming Language :: Python :: 3',
        'License :: OSI Approved :: MIT License',
    ],
    python_requires='>=3.6',
)
"""
with open("BARTScore/setup.py", "w") as f:
    f.write(setup_code)


In [32]:
!pip install ./BARTScore

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Processing ./BARTScore
  Preparing metadata (setup.py) ... done
  Created wheel for bart_score: filename=bart_score-0.1.0-py3-none-any.whl size=5317 sha256=2a3dbd4d92663b0b085bc62880bb303dae4166683ce00fa5ad1c98d9246df6c6
  Stored in directory: /tmp/pip-ephem-wheel-cache-y57o9zvq/wheels/98/01/d3/10d9ca6b2865ade955e5f3c526e4a9b480ab24cea364919527
Successfully built bart_score


In [9]:
import os
import json
import csv
import pickle
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from BARTScore.bart_score import BARTScorer
from bert_score import score as bert_score
import numpy as np

# --- Define Paths ---
json_folder_path = "/kaggle/input/new-batch/Batch_1"
image_base_path = "/kaggle/input/abo-small/small"
all_qa_data_path = "/kaggle/working/all_qa_data.pkl"
progress_file = "/kaggle/working/progress.json"

# from your_model_library import model, processor

# ---- LOAD CACHED DATA OR BUILD ----
if os.path.exists(all_qa_data_path):
    with open(all_qa_data_path, "rb") as f:
        all_qa_data = pickle.load(f)
else:
    print("🔄 Processing Q&A data from JSON files...")
    all_qa_data = []
    for json_file in os.listdir(json_folder_path):
        if not json_file.endswith(".json"):
            continue
        json_path = os.path.join(json_folder_path, json_file)
        with open(json_path, "r") as f:
            data = json.load(f)

        raw_path = data["image_path"].replace("\\", "/")
        path_parts = raw_path.split("/")[-2:]
        image_rel_path = "/".join(path_parts)
        full_img_path = os.path.join(image_base_path, image_rel_path)

        if not os.path.exists(full_img_path):
            continue

        try:
            raw_image = Image.open(full_img_path).convert("RGB")
        except Exception:
            continue

        for idx, qa_pair in enumerate(data.get("qa_pairs", [])):
            question = qa_pair.get("question", "").strip()
            answer = qa_pair.get("answer", "").strip()
            if question and answer:
                all_qa_data.append((json_file, idx, full_img_path, question, answer))

    with open(all_qa_data_path, "wb") as f:
        pickle.dump(all_qa_data, f)
    print("✅ Q&A data saved to disk.")

# ---- LOAD PROGRESS ----
if os.path.exists(progress_file):
    try:
        with open(progress_file, "r") as f:
            completed_set = set(tuple(x) for x in json.load(f))
        print(f"📘 Resuming from saved progress ({len(completed_set)} Q&A pairs done).")
    except json.JSONDecodeError as e:
        print(f"Error reading progress: {e}")
        completed_set = set()
else:
    completed_set = set()
    print("Progress file does not exist. Starting fresh.")

# ---- EVALUATION FUNCTION ----
def evaluate_qa_pairs(pairs, description, csv_filename):
    y_true, y_pred = [], []
    all_preds, all_gts = [], []
    bleu_scores, rougeL_scores, bart_scores = [], [], []

    rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    bart_scorer = BARTScorer(device="cuda", checkpoint="facebook/bart-large-cnn")

    with open(f"/kaggle/working/{csv_filename}", 'w', newline='') as csvfile:
        fieldnames = ['json_file', 'question_id', 'question', 'ground_truth', 'predicted_answer']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for json_file, idx, img_path, question, ground_truth in tqdm(pairs):
            if (json_file, idx) in completed_set:
                continue

            try:
                raw_image = Image.open(img_path).convert("RGB")
                inputs = processor(raw_image, question, return_tensors="pt").to("cuda")
                out = model.generate(**inputs)
                answer = processor.decode(out[0], skip_special_tokens=True)
            except Exception as e:
                print(f"[ERROR] {json_file} Q{idx+1} → {e}")
                answer = "ERROR"

            gt = ground_truth.strip().lower()
            pred = answer.strip().lower()
            y_true.append(gt)
            y_pred.append(pred)
            all_preds.append(pred)
            all_gts.append(gt)

            # BLEU
            bleu_scores.append(sentence_bleu([gt.split()], pred.split()))

            # ROUGE-L
            rougeL_scores.append(rouge.score(gt, pred)['rougeL'].fmeasure)

            # BARTScore
            bart_scores.append(bart_scorer.score([pred], [gt])[0])

            writer.writerow({
                'json_file': json_file,
                'question_id': idx,
                'question': question,
                'ground_truth': ground_truth,
                'predicted_answer': answer
            })

            completed_set.add((json_file, idx))
            with open(progress_file, "w") as f:
                json.dump(list(completed_set), f)

    # BERTScore (compute outside loop)
    print("📏 Calculating BERTScore...")
    bert_P, bert_R, bert_F1 = bert_score(all_preds, all_gts, lang='en', verbose=True)
    bert_P = P.numpy()
    bert_R = R.numpy()
    bert_F1 = F1.numpy()

    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average="macro")

    print(f"\n✅ Evaluation complete: {description}")
    print(f"Accuracy:       {acc:.4f}")
    print(f"F1 Score:       {f1:.4f}")
    print(f"BLEU:           {np.mean(bleu_scores):.4f}")
    print(f"ROUGE-L:        {np.mean(rougeL_scores):.4f}")
    print(f"BARTScore:      {np.mean(bart_scores):.4f}")
    print(f"BERTScore (P/R/F1): {np.mean(bert_P):.4f} / {np.mean(bert_R):.4f} / {np.mean(bert_F1):.4f}")

# ---- MAIN EXECUTION ----
filtered_qa_data = [item for item in all_qa_data if (item[0], item[1]) not in completed_set]
evaluate_qa_pairs(filtered_qa_data, "Full Dataset", "full_dataset_results.csv")

📘 Resuming from saved progress (573 Q&A pairs done).


  0%|          | 0/144321 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or

📏 Calculating BERTScore...


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

...

  0%|          | 0/231 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2256 [00:00<?, ?it/s]

done in 40.28 seconds, 3582.62 sentences/sec

✅ Evaluation complete: Full Dataset
Accuracy:       0.1901
F1 Score:       0.0152
BLEU:           0.0000
ROUGE-L:        0.2122
BARTScore:      -5.8426


TypeError: mean() received an invalid combination of arguments - got (dtype=NoneType, out=NoneType, axis=NoneType, ), but expected one of:
 * (*, torch.dtype dtype = None)
 * (tuple of ints dim, bool keepdim = False, *, torch.dtype dtype = None)
 * (tuple of names dim, bool keepdim = False, *, torch.dtype dtype = None)


In [11]:
import csv
from bert_score import score as bert_score
import numpy as np

# --- CONFIG ---
csv_path = "/kaggle/working/full_dataset_results.csv"  # or any result file
bert_lang = "en"  # language for BERTScore

# --- LOAD CSV DATA ---
ground_truths = []
predictions = []

print(f"📂 Reading predictions from {csv_path}...")
with open(csv_path, 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        gt = row['ground_truth'].strip()
        pred = row['predicted_answer'].strip()
        if gt and pred:
            ground_truths.append(gt)
            predictions.append(pred)

print(f"✅ Loaded {len(ground_truths)} Q&A pairs.")

# --- COMPUTE BERTSCORE ---
print("🔍 Computing BERTScore...")
P, R, F1 = bert_score(predictions, ground_truths, lang=bert_lang, verbose=True)
P = P.numpy()
R = R.numpy()
F1 = F1.numpy()
# --- OUTPUT RESULTS ---
print("\n🎯 BERTScore Results:")
print(f"Precision:  {np.mean(P):.4f}")
print(f"Recall:     {np.mean(R):.4f}")
print(f"F1 Score:   {np.mean(F1):.4f}")

# --- OPTIONAL: SAVE DETAILED SCORES ---
with open("/kaggle/working/bertscore_details.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Ground Truth", "Prediction", "Precision", "Recall", "F1"])
    for gt, pred, p, r, f1 in zip(ground_truths, predictions, P, R, F1):
        writer.writerow([gt, pred, f"{p:.4f}", f"{r:.4f}", f"{f1:.4f}"])

print("📁 Saved detailed BERTScore results to bertscore_details.csv")

📂 Reading predictions from /kaggle/working/full_dataset_results.csv...
✅ Loaded 144284 Q&A pairs.
🔍 Computing BERTScore...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/268 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2255 [00:00<?, ?it/s]

done in 43.18 seconds, 3341.14 sentences/sec

🎯 BERTScore Results:


TypeError: mean() received an invalid combination of arguments - got (dtype=NoneType, out=NoneType, axis=NoneType, ), but expected one of:
 * (*, torch.dtype dtype = None)
 * (tuple of ints dim, bool keepdim = False, *, torch.dtype dtype = None)
 * (tuple of names dim, bool keepdim = False, *, torch.dtype dtype = None)


In [12]:
P, R, F1 = bert_score(predictions, ground_truths, lang=bert_lang, verbose=True)
P = P.numpy()
R = R.numpy()
F1 = F1.numpy()
# --- OUTPUT RESULTS ---
print("\n🎯 BERTScore Results:")
print(f"Precision:  {np.mean(P):.4f}")
print(f"Recall:     {np.mean(R):.4f}")
print(f"F1 Score:   {np.mean(F1):.4f}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/268 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2255 [00:00<?, ?it/s]

done in 42.46 seconds, 3398.20 sentences/sec

🎯 BERTScore Results:
Precision:  0.9379
Recall:     0.9164
F1 Score:   0.9259


In [37]:
from BARTScore.bart_score import BARTScorer

bart_scorer = BARTScorer(device="cuda", checkpoint="facebook/bart-large-cnn")


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]